In [ ]:
import sys
import os.path as osp
import os
import sys
import random
import time
import glob
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data
 
from utils.ssd_model import VOCDataset, DataTransform, Anno_xml2list, od_collate_fn
from utils.ssd_model import SSD
from utils.ssd_model import MultiBoxLoss


In [ ]:
ROOT_PATH = 'YOUR_ROOT_PATH'
sys.path.append(ROOT_PATH)


In [ ]:
# データのリストを取得
data_path = 'YOUR_DATA_PATH'

filename_list = [os.path.split(f)[1].split('.')[0] for f in glob.glob(f'{data_path}/*.xml')]
filename_list_train, filename_list_val = train_test_split(filename_list, test_size=0.1)
train_img_list = [f'{data_path}/{f}.jpg' for f in filename_list_train]
train_anno_list = [f'{data_path}/{f}.xml' for f in filename_list_train]
val_img_list = [f'{data_path}/{f}.jpg' for f in filename_list_val]
val_anno_list = [f'{data_path}/{f}.xml' for f in filename_list_val]
 
# Datasetを作成
voc_classes = ['cop']
 
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 300  # 画像のinputサイズ
 
train_dataset = VOCDataset(train_img_list, 
                           train_anno_list, 
                           phase="train", 
                           transform=DataTransform(input_size, color_mean), 
                           transform_anno=Anno_xml2list(voc_classes))
val_dataset = VOCDataset(val_img_list, 
                         val_anno_list, 
                         phase="val", 
                         transform=DataTransform(input_size, color_mean), 
                         transform_anno=Anno_xml2list(voc_classes))
 
# DataLoaderを作成
train_dataloader = data.DataLoader(train_dataset, 
                                   batch_size=32, 
                                   shuffle=True, 
                                   collate_fn=od_collate_fn)
val_dataloader = data.DataLoader(val_dataset, 
                                 batch_size=3, 
                                 shuffle=False, 
                                 collate_fn=od_collate_fn)
 
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

In [ ]:
# SSD300の設定
ssd_cfg = {
    'num_classes': 2,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [21, 45, 99, 153, 207, 261],  # DBOXの大きさを決める
    'max_sizes': [45, 99, 153, 207, 261, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}
 
# SSDネットワークモデル
net = SSD(phase="train", cfg=ssd_cfg)


In [ ]:
# SSDの初期の重みを設定
vgg16_weight_path = 'VGG16_WEIGHT_PATH'
vgg_weights = torch.load(vgg16_weight_path)
net.vgg.load_state_dict(vgg_weights)


In [ ]:
# ssdのその他のネットワークの重みはHeの初期値で初期化
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # バイアス項がある場合
            nn.init.constant_(m.bias, 0.0)
        
# Heの初期値を適用
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)
 
# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
# 損失関数の設定
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)
 
# 最適化手法の設定
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)


In [ ]:
# モデルを学習させる関数を作成
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
 
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
 
    # ネットワークをGPUへ
    net.to(device)
 
    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True
 
    # イテレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0  # epochの損失和
    epoch_val_loss = 0.0  # epochの損失和
    min_loss = 9999
    logs = []
 
    # epochのループ
    for epoch in range(num_epochs+1):
 
        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()
 
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
 
            # データローダーからminibatchずつ取り出すループ
            with tqdm(dataloaders_dict[phase], desc=phase, file=sys.stdout) as iterator:
                for images, targets in iterator:
 
                    # GPUが使えるならGPUにデータを送る
                    images = images.to(device)
                    targets = [ann.to(device)
                               for ann in targets]  # リストの各要素のテンソルをGPUへ
 
                    # optimizerを初期化
                    optimizer.zero_grad()
 
                    # 順伝搬（forward）計算
                    with torch.set_grad_enabled(phase == 'train'):
                        # 順伝搬（forward）計算
                        outputs = net(images)
 
                        # 損失の計算
                        loss_l, loss_c = criterion(outputs, targets)
                        loss = loss_l + loss_c
 
                        # 訓練時はバックプロパゲーション
                        if phase == 'train':
                            loss.backward()
                            nn.utils.clip_grad_value_(net.parameters(), clip_value=2.0)
                            optimizer.step()
                            epoch_train_loss += loss.item()
                            iteration += 1
                        # 検証時
                        else:
                            epoch_val_loss += loss.item()
 
        # epochのphaseごとのlossと正解率
        t_epoch_finish = time.time()
        print(f'epoch {epoch+1}/{num_epochs} {(t_epoch_finish - t_epoch_start):.4f}sec || train_Loss:{epoch_train_loss:.4f} val_Loss:{epoch_val_loss:.4f}')
        t_epoch_start = time.time()
 
        # vallossが小さい、ネットワークを保存する
        if min_loss>epoch_val_loss:
            min_loss=epoch_val_loss
            trian_weight_path = 'YOUR_TRAIN_PATH'
            torch.save(net.state_dict(), trian_weight_path)
 
        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和
        